### Labic Images Segmentation - Main Notebook

#### Importando módulos

In [31]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport, PredictImages, DiceCoef
import tensorflow as tf

#### Diretórios e parâmetros

In [32]:
dataset_folder = "./TM40_Original"
norm_imgs_folder = "/Norm_images/*"
gt_folder = "/GT_images/*"

output_folder = "./TM40_46Prod"
test_imgs_folder = "/Producao/*"
gt_test_folder = "/GT_Producao/*"

ORIGINAL_SIZE = 180
NEW_SIZE = 128

n_fold = 2

batch_size = 4
epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)

#### Treinamento

##### Carregando dataset

In [33]:
imgs = Dataset(folder=dataset_folder, norm_imgs_folder=norm_imgs_folder, gt_folder=gt_folder, ORIGINAL_SIZE=ORIGINAL_SIZE, NEW_SIZE=NEW_SIZE)


Imagens carregadas com sucesso.


##### Dividindo dataset 

In [34]:
imgs.split_dataset()


Dataset subdividido.
20.0% dos dados para validação.
80.0% dos dados para treino.


##### Data Augmentation

In [35]:
data_aug = DataAugmentation(imgs.X_train, imgs.Y_train, imgs.X_val, imgs.Y_val)


Etapa de Data Augmentation concluída.


##### Treinamento da rede

In [36]:
segment = SegmentationModel(X_train=imgs.X_train, trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=epochs, callback=early_stopping)

Epoch 1/10
233/233 [==============================] - 12s 42ms/step - loss: 0.4735 - iou_score: 0.6107 - val_loss: 1.0066 - val_iou_score: 0.2414
Epoch 2/10
233/233 [==============================] - 9s 38ms/step - loss: 0.1557 - iou_score: 0.8639 - val_loss: 0.1220 - val_iou_score: 0.8915
Epoch 3/10
233/233 [==============================] - 9s 40ms/step - loss: 0.0928 - iou_score: 0.9182 - val_loss: 0.0777 - val_iou_score: 0.9316
Epoch 4/10
233/233 [==============================] - 9s 39ms/step - loss: 0.0641 - iou_score: 0.9433 - val_loss: 0.0748 - val_iou_score: 0.9335
Epoch 5/10
233/233 [==============================] - 10s 41ms/step - loss: 0.0542 - iou_score: 0.9526 - val_loss: 0.0722 - val_iou_score: 0.9368
Epoch 6/10
233/233 [==============================] - 11s 46ms/step - loss: 0.0529 - iou_score: 0.9538 - val_loss: 0.0656 - val_iou_score: 0.9414
Epoch 7/10
233/233 [==============================] - 11s 47ms/step - loss: 0.0433 - iou_score: 0.9619 - val_loss: 0.0713 - val

##### Salvando modelo

In [37]:
save_report = SaveReport(model=segment.model, folder_name=output_folder, n_fold=n_fold, use_batch_size=batch_size, epochs=epochs)

Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390  ja existe
Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2  Criado 

Modelo salvo.
Nome: model_4_10_exec_2023-04-17-09-45-24.444390_fold_2
Salvo em: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390


#### Predição 

##### Carregando imagens de teste

In [38]:
test_imgs = Dataset(folder=output_folder, norm_imgs_folder=test_imgs_folder, gt_folder=gt_test_folder, ORIGINAL_SIZE=ORIGINAL_SIZE, NEW_SIZE=NEW_SIZE)


Imagens carregadas com sucesso.


#### Segmentando imagens de teste

In [39]:
prediction = PredictImages(test_images=test_imgs, n_fold_folder_name=save_report.n_fold_folder_name, \
                           model_name=save_report.model_name, use_batch_size=batch_size, img_shape=test_imgs.img_shape)

Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod  Criado 


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_025_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_026_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_027_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7


Imagens preditas com sucesso.


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_423_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_424_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:238: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-17-09-45-24.444390/fold_2/outputs_prod/predicao_425_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7

##### Salvando DSC

In [40]:
dice = DiceCoef(gt_imgs=test_imgs.Y, pred_folder=prediction.n_fold_folder_name+"/outputs_prod/*", new_size=NEW_SIZE)
dice.save_dice(f"{save_report.n_fold_folder_name}/dice_fold_{n_fold}.txt")

Coeficiente Dice: 0.9382552600425175
